# TP2: Classificação de Imagens com Banco de Filtros 

<b>Alunos</b>: Rafael Gurgel + Rafael Martins de Souza<br>
               

Neste trabalho iremos classificar dígitos da base de dados MNIST, formada por um conjunto de imagens 28x28 contendo dígitos manuscritos. Em particular, desenvolvemos um classificador que é capaz de distinguir o número 3 do número 8.   

# Imports

In [38]:
%matplotlib inline 

import numpy as np
import os
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
import skimage.io as io

# Carregando as imagens

In [96]:
import random
from skimage.transform import resize

imgs = []

for i in [3, 8]:
    for fn in os.listdir('MNIST/' + str(i)):
        img = io.imread('MNIST/%d/%s' % (i, fn), dtype=np.float64)
        img = resize(img, (29, 29))
        img = img / 255
        imgs.append((img, 1 if i == 8 else 0))

random.shuffle(imgs)

/home/rafael/Apps/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


O código acima carrega o dataset em uma lista <b>imgs</b>. 

Realizamos certas transformações nas imagens para facilitar os próximos passos. 
Em particular, transformamos a imagem de 28x28 para 29x29 e a normalizamos, fazendo com que os valores de cada pixel fiquem entre 0 e 1.

Ao final, as imagens são randomizadas.

# Classificador

In [59]:
def predict(img, model):
    F1, F2, w = model
    M2 = correlate(img, F1, mode='valid')
    v1 = correlate(M2, F2, mode='valid')[0][0]
    v2 = v1 * w[0]
    r = expit(v2)
    return 1 if r > 0.5 else 0

O classificador funciona de maneira bem simples. A imagem <b>img</b> passada como parâmetro, de tamanho 29x29 passa por um filtro 15x15 <b>F1</b>, resultando em uma imagem 15x15. Por sua vez, essa imagem obtida passa por um outro filtro 15x15 <b>F2</b>, resultando em um valor <b>v1</b>. Ele é multiplicado por um peso <b>w[0]</b> e aplicado a função sigmoid (<b>expit</b>). Caso o resultado obtido seja maior que 0.5, o número será classificado como 8. Caso contrário, será classificado como 3.

In [101]:
from IPython.display import clear_output

def train_dataset(imgs):
    F1 = (np.random.rand(15,15) - 0.5) * 0.1
    F2 = (np.random.rand(15,15) - 0.5) * 0.1
    w = (np.random.rand(1) - 0.5) * 0.1
    correct = 0
    
    for i, (img, cls) in enumerate(imgs):
        pred = train_step(img, F1, F2, w, cls, 0.1)
        if pred == cls:
            correct += 1
        
        clear_output(wait=True)
        print("%d/%d => %.1f%%" % (i+1, len(imgs), 100 * correct / (i+1)))
        
    return (F1, F2, w)

A função acima inicializa um modelo aleatório e o treina. Os filtros <b>F1</b>, <b>F2</b> e o peso <b>w[0]</b> são inicializados com valores aleatórios entre -0.05 e 0.05. Ela retorna os filtros e pesos otimizados. 

Estamos utilizando como método de otimização o Stochastic Gradient Descent (SGD) com batches de tamanho 1. Para cada imagem, realizamos um passo do SGD, que é representado pela função <b>train_step</b>, que é definida abaixo: 

In [110]:
import math
from scipy.special import expit
from scipy.signal import correlate

def grad_F1(M1, gM2):
    return correlate(M1, gM2)

def grad_M2(dE_dv1, F2):
    return dE_dv1 * F2

def grad_F2(dE_dv1, M2):
    return dE_dv1 * M2
    
def train_step(M1, F1, F2, w, C, lr):
    M2 = correlate(M1, F1, mode='valid')
    v1 = correlate(M2, F2, mode='valid')[0][0]
    v2 = v1 * w[0]
    r = expit(v2)
    
    E = (r - C)**2
    
    dE_dv2 = (r - C) * r * (1 - r)
    dE_dw = dE_dv2 * v1
    dE_dv1 = dE_dv2 * w[0]
    
    gF2 = grad_F2(dE_dv1, M2)
    gM2 = grad_M2(dE_dv1, F2)
    gF1 = grad_F1(M1, gM2)
    
    w[0] -= lr * dE_dw
 
    for i in np.ndindex(F1.shape):
        F1[i] -= lr * gF1[i]
        
    for i in np.ndindex(F2.shape):
        F2[i] -= lr * gF2[i]
        
    return 1 if r > 0.5 else 0

A função <b>train_step</b> realiza um passo do SGD. Ela recebe, como parâmetros, uma matriz 29x29 <b>M1</b>, que representa uma imagem, dois filtros 15x15 (<b>F1</b> e <b>F2</b>), um peso <b>w</b>, a classe da imagem <b>C</b> (que é 1 se a imagem representar um 8, e 0 se representar um 3) e a taxa de aprendizado <b>lr</b>.  

No corpo da função, primeiramente tentamos classificar a imagem como 8 ou 3, de maneira similar a função <b>predict</b>. Obtemos como resultado <b>r</b>. Quanto mais perto <b>r</b> estiver de 1, mais confiante o classificador está de que a imagem representa o número 8, e, de maneira análoga, quanto mais perto de 0, mais confiante ele está de que a imagem representa o número 3.  

Uma vez que temos a predição do classificador, computamos o seu erro <b>E</b>: 

$E = (r - C)^2$. 

Queremos minimizar esse erro. Para tanto, calculamos o gradiente do erro <b>E</b> com relação aos diversos parâmetros do modelo - em particular, <b>w[0]</b> e as células de <b>F1</b> e <b>F2</b>. Os gradientes do erro com relação aos parâmetros do modelo estão armazenados nas variáveis <b>dE_dw</b>, <b>gF1</b> e <b>gF2</b>, respectivamente. Uma vez obtidos os gradientes, multiplicamos eles pela taxa de aprendizado <b>lr</b> e os subtraímos dos parâmetros. 

# Testes

Utilizaremos 80% das imagens como treino e 20% como validação. A acurácia será medida através da expressão:

$\frac{TP + TN}{TP + FN + FP + TN}$

onde

TP = True Positive<br/>
TN = True Negative<br/>
FN = False Negative<br/>
TN = True Negative<br/>

Iremos considerar o valor 8 (classe 1) como positive, e o valor 3 (classe 0) como negative.

In [256]:
from IPython.display import clear_output

train_size = int(0.8 * len(imgs))
train = imgs[:train_size]
test = imgs[train_size:]

model = train_dataset(train)
correct = 0

for i, (img, cls) in enumerate(test):
    pred = predict(img, model)
    if pred == cls: correct += 1
    clear_output(wait=True)
    print("Validando: %d/%d\n" % (i+1, len(test)))
    
clear_output(wait=True)
print("Acurácia no teste: %.1f%%" % (100 * correct / len(test)))

Acurácia no teste: 92.4%
